In [115]:

import spacy

In [32]:
spacy.cli.download("en_core_web_sm")

# Load the English model
nlp = spacy.load("en_core_web_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.1 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
from zipfile import ZipFile
with ZipFile('movie_conversations.txt.zip','r') as z1:
    z1.extractall()
with ZipFile('movie_lines.txt.zip','r') as z2:
    z2.extractall()

In [4]:
corpmovie1='movie_conversations.txt'
with open(corpmovie1,'r') as corp1:#conversation numbered
    convreader=corp1.readlines()

In [14]:
convreader[0]

"u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']\n"

In [9]:
lines='movie_lines.txt'
with open(lines,'r',errors='replace') as line1:
    linereader=line1.readlines()

In [15]:
linereader[0:2]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n']

In [17]:
#use hashmap to map from lines to  lineid to line
linemap={}
for val in linereader:
    x=val.split('+++$+++')
    linemap[x[0]]=x[-1]

In [21]:
linemap[next(iter(linemap))] #example of the value of a sample line of this map

' They do not!\n'

In [ ]:
text = "Hello, how are you today?"  #delete this

# Tokenize the text and remove punctuation
tokens = [token.text for token in nlp('HOW ARE YOU') if not token.is_punct]

# Print the tokens without punctuation
print(tokens)

In [59]:
linemap = {key.strip(): value for key, value in linemap.items()}
linemap['L134']

" But imagine the things he'd say during sex.\n"

['be', 'you', 'my', 'attorney', 'I', 'be', 'Emil', 'I', 'be', 'insane']

In [62]:
qaarray=[]
from ast import literal_eval
for val in convreader:
    num=literal_eval(val.split('+++$+++')[-1])
    for x in range(len(num)):
        pair=[]
        if x !=len(num)-1:
            pair.append([token.lemma_ for token in nlp(linemap[num[x]]) if not token.is_space and not token.is_punct])
            pair.append([token.lemma_ for token in nlp(linemap[num[x+1]]) if not token.is_space and not token.is_punct])
            qaarray.append(pair)


In [66]:
qaarray[0][1]

['well',
 'I',
 'think',
 'we',
 'would',
 'start',
 'with',
 'pronunciation',
 'if',
 'that',
 'be',
 'okay',
 'with',
 'you']

In [68]:
#wordtoindex
from collections import Counter
wordfrequencycounter=Counter()
for x in qaarray:
    wordfrequencycounter.update(x[0])#update all question
    wordfrequencycounter.update(x[1])#update all answers

In [136]:
#apply min word freq mask
min_words=6
allwords=[word for word in wordfrequencycounter.keys() if wordfrequencycounter[word]>min_words]
wordmap={key:val+1 for val,key in enumerate(allwords)}

In [137]:
wordmap #add unknown tokens for words occuring less than 4

{'can': 1,
 'we': 2,
 'make': 3,
 'this': 4,
 'quick': 5,
 'and': 6,
 'Andrew': 7,
 'Barrett': 8,
 'be': 9,
 'have': 10,
 'an': 11,
 'incredibly': 12,
 'public': 13,
 'break-': 14,
 'up': 15,
 'on': 16,
 'the': 17,
 'again': 18,
 'well': 19,
 'I': 20,
 'think': 21,
 'would': 22,
 'start': 23,
 'with': 24,
 'if': 25,
 'that': 26,
 'okay': 27,
 'you': 28,
 'not': 29,
 'gag': 30,
 'spit': 31,
 'part': 32,
 'please': 33,
 'then': 34,
 'how': 35,
 "'bout": 36,
 'try': 37,
 'out': 38,
 'some': 39,
 'french': 40,
 'cuisine': 41,
 'Saturday': 42,
 'Night': 43,
 'ask': 44,
 'so': 45,
 'cute': 46,
 'what': 47,
 'your': 48,
 'name': 49,
 'forget': 50,
 'it': 51,
 'no': 52,
 'my': 53,
 'fault': 54,
 'do': 55,
 'a': 56,
 'proper': 57,
 'introduction': 58,
 'Cameron': 59,
 'thing': 60,
 'at': 61,
 'mercy': 62,
 'of': 63,
 'particularly': 64,
 'hideous': 65,
 'breed': 66,
 'loser': 67,
 'sister': 68,
 'date': 69,
 'until': 70,
 'she': 71,
 'seem': 72,
 'like': 73,
 'could': 74,
 'get': 75,
 'easy': 7

In [138]:
len(wordmap)

16081

In [146]:
wordmap['<unk>']=len(wordmap)+1
wordmap['<pad>']=0
wordmap['<start>']=len(wordmap)+1
wordmap['<end>']=len(wordmap)+1

In [147]:
#export json dictionary
import json
with open('wordmap.json','w') as j1:
    json.dump(wordmap,j1)

In [265]:
#now tokenize question and answers using our wordmap
def tokenizequestion(words,wordmap,maxlenpad=25):
    if len(words)>=maxlenpad:
        tokenized=[wordmap.get(string,wordmap['<unk>']) for string in words[:maxlenpad-len(words)]]
    else:
        tokenized=[wordmap.get(string,wordmap['<unk>']) for string in words]+[wordmap['<pad>']]*(maxlenpad-len(words))
    return tokenized
def tokenizeanswer(words,wordmap,maxlenpad):
    if len(words)>=maxlenpad:
                tokenized=[wordmap.get(string,wordmap['<unk>']) for string in words[:maxlenpad-len(words)]]
                tokenized=[wordmap['<start>']]+tokenized+[wordmap['<end>']]
    else:
        tokenized=[wordmap['<start>']]+[wordmap.get(string,wordmap['<unk>']) for string in words]+[wordmap['<end>']]+[wordmap['<pad>']]*(maxlenpad-len(words))
    return tokenized

In [266]:
maxlenpad=25
print((qaarray[90][1]))


['why', 'do', 'not', 'you', 'tell', 'I']


In [267]:
print((tokenizequestion(qaarray[0][0],wordmap,maxlenpad)))

[1, 2, 3, 4, 5, 16086, 16086, 6, 7, 8, 9, 10, 11, 12, 16086, 13, 14, 15, 16, 17, 16086, 18, 0, 0, 0]


In [268]:
#loop over all qaarray values
tokenizedvals=[] #all q and a tokenized
for val in qaarray:
    q=tokenizequestion(val[0],wordmap,maxlenpad)
    a=tokenizeanswer(val[1],wordmap,maxlenpad)
    if len(q) and len(a)!=2:
        tokenizedvals.append([q,a])

In [269]:
#DONE PREPROCESSING.EXPORT THIS
with open('tokenizedpair.json','w') as write:
    json.dump(tokenizedvals,write)

In [239]:
len(tokenizedvals[25][1])

27

In [259]:
len(tokenizedvals)

218338

In [264]:
for x in range(218338):
    if(len((tokenizedvals[x][0]))!=25):
        print(x)

In [243]:
len(tokenizedvals[0][1])

27

In [244]:
len(tokenizedvals)

218338

In [254]:
len(tokenizedvals[46][0])

25

In [273]:
import numpy as np
np.asarray(tokenizedvals)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (218338, 2) + inhomogeneous part.

In [275]:
with open('pairs.json','w') as pairs:
    json.dump(qaarray,pairs)